The CTD for the TYZ5 10-Year U.S. Treasury Note futures is the T 3.875 09/30/32 Govt Bond, because it has the lowest net basis among all eligible deliverables.

In [1]:
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
# --- Inputs (Bloomberg data) ---
bonds = [
    ("T 2.75 08/15/32 Govt", 93.203125, 2.75, "8/15/2032", 0.403532609),
    ("T 4.125 11/15/32 Govt", 101.390625, 4.125, "11/15/2032", 1.636548913),
    ("T 3.5 02/15/33 Govt", 97.328125, 3.5, "2/15/2033", 0.513586957),
    ("T 3.375 05/15/33 Govt", 96.3125, 3.375, "5/15/2033", 1.338994565),
    ("T 3.875 08/15/33 Govt", 99.421875, 3.875, "8/15/2033", 0.56861413),
    ("T 4.5 11/15/33 Govt", 103.609375, 4.5, "11/15/2033", 1.785326087),
    ("T 4 06/30/32 Govt", 100.734375, 4.0, "6/30/2032", 1.086956522),
    ("T 4 07/31/32 Govt", 100.703125, 4.0, "7/31/2032", 0.75),
    ("T 3.875 08/31/32 Govt", 99.921875, 3.875, "8/31/2032", 0.406767956),
    ("T 3.875 09/30/32 Govt", 99.875, 3.875, "9/30/2032", 0.085164835),
]

F = 112.67188                 # TYZ5 futures price
DELIVERY = date(2025, 12, 1)  # Delivery date
HALF_YEAR_YIELD = 0.06 / 2    # 6% annual -> 3% per half-year (for CF)

In [ ]:
# --- Date helpers ---
def end_of_month(d: date) -> date:
    """Last day of the month of d (handles 30/31/Feb cases)."""
    first_next = date(d.year, d.month, 1) + relativedelta(months=1)
    return first_next - timedelta(days=1)

def align_day(y: int, m: int, day: int) -> date:
    """Return y-m-day; if invalid, use month end (preserves EOM behavior)."""
    try:
        return date(y, m, day)
    except ValueError:
        return end_of_month(date(y, m, 1))

def last_and_next_coupon(as_of: date, maturity: date):
    """US Treasury coupons: every 6 months on the maturity day-of-month."""
    m, d = maturity.month, maturity.day
    t = align_day(as_of.year, m, d)
    while t > as_of:                        # step back to find last coupon <= as_of
        prev = t - relativedelta(months=6)
        t = align_day(prev.year, prev.month, d)
    last_c = t
    nxt = last_c + relativedelta(months=6)  # next coupon after as_of
    next_c = align_day(nxt.year, nxt.month, d)
    return last_c, next_c

# --- Accrued interest at delivery (ACT/ACT Treasury) ---
def ai_at_delivery(coupon_pct: float, maturity: date) -> float:
    """AI_D = fraction of period × semiannual coupon (per $100)."""
    semi_coupon = coupon_pct / 2.0
    last_c, next_c = last_and_next_coupon(DELIVERY, maturity)
    frac = (DELIVERY - last_c).days / (next_c - last_c).days
    return semi_coupon * frac

# --- Conversion factor at 6% (semiannual compounding) ---
def conversion_factor(coupon_pct: float, maturity: date):
    """
    CF = (PV at 6% semiannual of remaining coupons and principal − AI_D) / 100.
    Discount uses integer half-years k = 1..N with rate 3% per half-year.
    """
    semi_coupon = coupon_pct / 2.0
    _, next_c = last_and_next_coupon(DELIVERY, maturity)

    # Build remaining payment dates from next coupon to maturity
    pays, d = [], next_c
    while d <= maturity:
        pays.append(d)
        step = d + relativedelta(months=6)
        d = align_day(step.year, step.month, maturity.day)

    N = len(pays)
    pv_dirty = sum(semi_coupon / ((1 + HALF_YEAR_YIELD) ** k) for k in range(1, N + 1))
    pv_dirty += 100.0 / ((1 + HALF_YEAR_YIELD) ** N)

    aiD = ai_at_delivery(coupon_pct, maturity)
    cf = (pv_dirty - aiD) / 100.0
    return cf, aiD

In [4]:
rows = []
for name, p_clean, cpn, mat_str, ai0 in bonds:
    mat = pd.to_datetime(mat_str).date()
    cf, aiD = conversion_factor(cpn, mat)

    p_dirty = p_clean + ai0                   # today’s dirty price
    invoice = F * cf + aiD                    # invoice price at delivery
    net_basis = p_dirty - invoice             # lower = cheaper to deliver

    rows.append([
        name, cpn, mat,
        round(cf, 6),
        round(p_clean, 6), round(ai0, 6), round(p_dirty, 6),
        round(aiD, 6), round(invoice, 6), round(net_basis, 6)
    ])

In [5]:
out = pd.DataFrame(rows, columns=[
    "Bond", "Coupon (%)", "Maturity", "Conversion Factor",
    "P_clean,0", "AI_0", "P_dirty,0", "AI_D", "Invoice Price", "Net Basis"
]).sort_values("Net Basis").reset_index(drop=True)

out

,Bond,Coupon (%),Maturity,Conversion Factor,"P_clean,0",AI_0,"P_dirty,0",AI_D,Invoice Price,Net Basis
0,T 3.875 09/30/32 Govt,3.875,2032-09-30,0.873342,99.875000,0.085165,99.960165,0.663674,99.064813,0.895351
1,T 3.875 08/31/32 Govt,3.875,2032-08-31,0.870131,99.921875,0.406768,100.328643,0.984807,99.024120,1.304523
2,T 4 07/31/32 Govt,4.000,2032-07-31,0.873670,100.703125,0.750000,101.453125,1.336957,99.774965,1.678160
3,T 2.75 08/15/32 Govt,2.750,2032-08-15,0.808368,93.203125,0.403533,93.606658,0.807065,91.887425,1.719232
4,T 4 06/30/32 Govt,4.000,2032-06-30,0.870209,100.734375,1.086957,101.821332,1.683060,99.731107,2.090225
5,T 4.125 11/15/32 Govt,4.125,2032-11-15,0.892276,101.390625,1.636549,103.027174,0.182320,100.716747,2.310427
6,T 3.875 08/15/33 Govt,3.875,2033-08-15,0.855166,99.421875,0.568614,99.990489,1.137228,97.490390,2.500099
7,T 3.5 02/15/33 Govt,3.500,2033-02-15,0.815044,97.328125,0.513587,97.841712,2.794199,94.626765,3.214947
8,T 4.5 11/15/33 Govt,4.500,2033-11-15,0.903803,103.609375,1.785326,105.394701,0.198895,102.032054,3.362647
9,T 3.375 05/15/33 Govt,3.375,2033-05-15,0.816793,96.312500,1.338995,97.651495,1.834239,93.863858,3.787636
